# 005: Cross Validate Models

In [ ]:
import os
import sys

import numpy as np

sys.path.append("../")
from models import OrdinaryLeastSquares, LogisticRegression, LinearSVM, KNearestNeighbors
from model_selection import cross_validation
from preprocessing import preprocess, normalize_and_bias_data

%load_ext autoreload
%autoreload 2

def print_results(cv_results):
    print(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}")
    print(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}")
    print(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}")
    print(f" & {np.mean(cv_results.f1_scores)*100:.1f}±{np.std(cv_results.f1_scores)*100:.1f} & "
          f"{np.mean(cv_results.f2_scores)*100:.1f}±{np.std(cv_results.f2_scores)*100:.1f} & "
          f"{np.mean(cv_results.auc_rocs)*100:.1f}±{np.std(cv_results.auc_rocs)*100:.1f}")

In [3]:
x_train, _, y_train, *_ = preprocess(save_dir="../data/dataset_prep")
x_train.shape

Loading raw data...
Replacing missing value codes with np.nan...
Applying one-hot encoding...
Saving preprocessed data to ../data/dataset_prep...


(328135, 622)

In [2]:
train = np.load("../data/dataset_prep/train.npz")
x_train, y_train = train["x_train"], train["y_train"]
x_train.shape

(328135, 321)

In [11]:
x_train, _, y_train, *_ = preprocess(one_hot_encoding=False)

Replacing missing value codes with np.nan...


In [13]:
model_settings = [
    #{"model_class": OrdinaryLeastSquares},
    #{"model_class": LogisticRegression, "gamma": 1e-1},
    {"model_class": LinearSVM},
    #{"model_class": KNearestNeighbors},
]
for model in model_settings:
    print(f"Cross-validating model: {model['model_class'].__name__}")
    num_samples = int(1e5) if model['model_class'] != KNearestNeighbors else int(1e5)
    cv_results = cross_validation(x_train[:num_samples], y_train[:num_samples], verbose=True, **model)
    print_results(cv_results)
    if num_samples != 1e6: break
    out_dir = "../results"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{model['model_class'].__name__}.txt")

    with open(out_path, "w") as f:
        f.write(f"Model: {model['model_class'].__name__}\n\n")
        f.write("cv_results repr:\n")
        f.write(repr(cv_results))
        f.write("\n\n")
        f.write("Aggregated metrics:\n")
        f.write(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}\n")
        f.write(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}\n")
        f.write(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}\n")
    

Cross-validating model: LinearSVM
Starting fold 1/5 with 80000 samples
Iteration 0, Training Loss: 1.0, Validation Loss: 1.0
Iteration 100, Training Loss: 0.9442201023753964, Validation Loss: 0.9451980027752828
Iteration 200, Training Loss: 0.936689514988119, Validation Loss: 0.9372818248793686
Iteration 300, Training Loss: 0.9341161105310835, Validation Loss: 0.9345789244272138
Iteration 400, Training Loss: 0.9329074638290952, Validation Loss: 0.9333286632557469
Iteration 500, Training Loss: 0.9322472404970871, Validation Loss: 0.9326594293336611
Iteration 600, Training Loss: 0.9318230746220649, Validation Loss: 0.9322369505624529
Iteration 700, Training Loss: 0.9315318426137287, Validation Loss: 0.9319496666603637
Iteration 800, Training Loss: 0.9313263231078395, Validation Loss: 0.9317492393486827
Iteration 900, Training Loss: 0.9311810151510108, Validation Loss: 0.9316085809625748
Iteration 0, Training Loss: 1.0, Validation Loss: 1.0
Iteration 100, Training Loss: 0.9310581932156438

KeyboardInterrupt: 